In [2]:
import pandas as pd
import numpy as np

def shot_after_DTL(player, path, side):
    # Load the data
    events = pd.read_csv(path)
    events['pointWonBy'] = events.groupby('pointNumber')['pointWonBy'].bfill()

    # Filter for rows where the shot was a "Forehand, Down the Line, Deuce" shot in the same point
    forehand_dtl_deuce = events[(events['shotFhBh'] == 'Forehand') & 
                                (events['shotDirection'] == 'Down the Line') & 
                                (events['side'] == side)]

    # Initialize list for valid indices of the shots after "Forehand, Down the Line, Deuce"
    target_indices = []
    for idx in forehand_dtl_deuce.index:
        # Ensure idx + 1 is within bounds and that the next shot belongs to the same point
        if idx + 1 in events.index and events.loc[idx + 1, 'pointNumber'] == events.loc[idx, 'pointNumber']:
            target_indices.append(idx + 1)
    
    # Filter `events` for the player's returns using valid target indices
    returns_place = events.loc[target_indices]
    returns_place = returns_place[returns_place['shotHitBy'] == player].copy()

    # Adjust shotLocationY and shotLocationX
    returns_place['shotLocationY'] = returns_place['shotLocationY'].apply(lambda y: -y if y > 0 else y)
    returns_place['shotLocationX'] = returns_place.apply(lambda row: -row['shotLocationX'] if row['shotLocationY'] < 0 else row['shotLocationX'], axis=1)

    # Export the data as JSON
    returns_place_json = returns_place.to_json(orient='records')

    with open('after_DTL.json', 'w') as f:
        f.write(returns_place_json)
    
# Run function with given input
player = "Rudy Quan"
path = '../../Shot_Visuals_RudyQuan_AristotelisThanos.csv'
side = "Deuce"
shot_after_DTL(player, path, side)

